<a href="https://colab.research.google.com/github/w2k31984/tripleten_Bootcamp_DS/blob/master/03_business_decisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decisiones de negocio

Habiendo analizado los datos, ahora pondremos en contexto la decisión final que queremos tomar ¿cuál canción navideña deberíamos de escoger para nuestro anuncio de marketing?

Para tomar esta decisión necesitamos más contexto como:
- el costo por canción
- las ganancias esperadas por haber usado la canción

Utilizaremos los mismos datos que usamos ayer y agregaremos otros que nos hablarán sobre el costo y nuestra expectativa de ingresos por cada canción.

Pueden leer más acerca de los datos en las siguientes ligas:
- [Billboard Top 100 Christmas Carol Dataset](https://www.kaggle.com/datasets/sharkbait1223/billboard-top-100-christmas-carol-dataset/)
- [Billboard Hot weekly charts](https://data.world/kcmillersean/billboard-hot-100-1958-2017)

Los datos los puedes descargar desde aquí:
- [christmas_billboard_data.csv](https://drive.google.com/file/d/1uDsTeUOXmyuhRbQ_5tveNHN3sTWtYG0B/view?usp=sharing)


In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from numpy.random import lognormal, normal, uniform

sns.set_theme()

In [4]:
url = 'https://raw.githubusercontent.com/HectorLira/data-science-projects/master/datasets/christmas_billboard_data.csv'
carols = pd.read_csv(url)
print("Completado con exito")

Completado con exito


In [6]:
carols.head(5)

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,month,day
0,http://www.billboard.com/charts/hot-100/1958-1...,12/13/1958,83,RUN RUDOLPH RUN,Chuck Berry,Run Rudolph RunChuck Berry,1,NaN,69,3,1958,12,13
1,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,57,JINGLE BELL ROCK,Bobby Helms,Jingle Bell RockBobby Helms,1,NaN,29,19,1958,12,20
2,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,73,RUN RUDOLPH RUN,Chuck Berry,Run Rudolph RunChuck Berry,1,83.0,69,3,1958,12,20
3,http://www.billboard.com/charts/hot-100/1958-1...,12/20/1958,86,WHITE CHRISTMAS,Bing Crosby,White ChristmasBing Crosby,1,NaN,12,13,1958,12,20
4,http://www.billboard.com/charts/hot-100/1958-1...,12/27/1958,44,GREEN CHRI$TMA$,Stan Freberg,Green Chri$tma$Stan Freberg,1,NaN,44,2,1958,12,27


In [7]:
unique_carols = carols.groupby("songid")["weeks_on_chart"].max().reset_index()

## Datos adicionales

En la realidad, los siguientes datos vendrían de otro análisis o algún modelo predictivo. Por ahora, los simularemos usando código.

In [8]:
np.random.seed(100)

carols_profit = pd.DataFrame(data={
    "songid": carols["songid"].unique(),
    "cost": lognormal(size=len(carols["songid"].unique())) * 1000,
    "uncertainty": uniform(size=len(carols["songid"].unique()))
})

carols_profit["expected_revenue"] = carols_profit["cost"] + (normal(size=len(carols["songid"].unique())) * 100)

In [9]:
carols_profit.head()

,songid,cost,uncertainty,expected_revenue
0,Run Rudolph RunChuck Berry,173.814703,0.200124,23.056187
1,Jingle Bell RockBobby Helms,1408.718468,0.657625,1419.506881
2,White ChristmasBing Crosby,3167.795128,0.778289,3242.500693
3,Green Chri$tma$Stan Freberg,776.905905,0.779598,819.873548
4,"The Happy ReindeerDancer, Prancer And Nervous",2667.977746,0.610328,2526.473454


# Tomando una decisión

Una decisión de negocio debe de incorporar todos los elementos necesarios para estar completamente convencidos de que es la mejor decisión. Al final, no es posible tener información completa del futuro y de cómo se van a comportar las cosas pero tomamos las decisiones con la mayor cantidad de información posible.

In [10]:
# Calculemos los beneficios esperados de cada canción (usen la columna "expected_revenue" y "cost")
carols_profit["profit"] = carols_profit["expected_revenue"] - carols_profit["cost"]

carols_profit_with_info = (
    carols
    .merge(carols_profit, on="songid")
)

(
    carols_profit_with_info
    .query("(year < 2020) and (year >= 1980) and (weeks_on_chart > 7)")
    .sort_values(by="profit", ascending=False)[["songid", "weeks_on_chart", "peak_position", "profit", "cost", "expected_revenue"]]
    .drop_duplicates()
    .head(20)
)

,songid,weeks_on_chart,peak_position,profit,cost,expected_revenue
88,Rockin' Around The Christmas TreeBrenda Lee,18,14,207.479317,827.376166,1034.855483
259,This One's For The ChildrenNew Kids On The Block,16,7,199.027365,291.581853,490.609218
294,All I Want For Christmas Is YouMariah Carey,19,11,123.690789,199.173594,322.864383
326,BelieveBrooks & Dunn,20,60,73.699517,390.609804,464.309321
230,Same Old Lang SyneDan Fogelberg,18,9,30.104946,2117.941786,2148.046733
19,Jingle Bell RockBobby Helms,19,29,10.788413,1408.718468,1419.506881
248,Do They Know It's Christmas?Band-Aid,9,13,-12.620118,257.586666,244.966548
71,The Christmas Song (Merry Christmas To You)Nat...,8,38,-43.812092,342.993655,299.181563
322,Better DaysGoo Goo Dolls,20,36,-59.137512,577.096263,517.958750
363,MistletoeJustin Bieber,10,11,-90.703042,351.969809,261.266767


from matplotlib import pyplot as plt
_df_0['weeks_on_chart'].plot(kind='hist', bins=20, title='weeks_on_chart')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['peak_position'].plot(kind='hist', bins=20, title='peak_position')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['profit'].plot(kind='hist', bins=20, title='profit')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['cost'].plot(kind='hist', bins=20, title='cost')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='cost', y='expected_revenue', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['weeks_on_chart'].plot(kind='line', figsize=(8, 4), title='weeks_on_chart')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['peak_position'].plot(kind='line', figsize=(8, 4), title='peak_position')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['profit'].plot(kind='line', figsize=(8, 4), title='profit')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['cost'].plot(kind='line', figsize=(8, 4), title='cost')
plt.gca().spines[['top', 'right']].set_visible(False)

# Conclusión

- La decisión final será la de usar la canción de `Rockin' Around The Christmas Tree` de `Brenda Lee`.
- La decisión la tomamos basándonos en:
   - La canción cumple con las condiciones de nustra preferencia (canción lanzada depsués de 1980 y que se haya mantenido en el Billboard por más de 7 semanas)
   - Se maximiza la utilidad esperada neta (ingresos esperados - costos)
   - (Contexto adicional) A pesar de que no es la canción con mayor utilidad esperada neta, es una canción que lleva más semanas en el Billboard y sabemos (como expertos en música) que `Brenda Lee` es una artista más popular que los otros dos artistas (`Bobby Darin` y `Chris Brown`).